In [89]:
import random
import math

class KnapsackGenetic:
  def __init__(self, params):
    self.ALL_NUMBERS = list(range(params["max_per_item"] + 1)) #Create a list [0,...n-1]. if n = 3 then [0,1,2,3]
    self.params = params
    self.specimen = [None] * self.params["generation_size"] # [None, None, None, None] if generation_size = 4

    self.create_initial_population()

  def create_initial_population(self):
    self.specimen = list(map(
        lambda _: list(map(
            lambda _: random.choice(self.ALL_NUMBERS), # If n = 3 then choose some number from [0,1,2,3]
            [None] * len(self.params["items"]) # Here len = 5, there are 5 items. [None, None, None, None, None]
        )                                      # So, it has been chosen some random specimen example: [0,1,3,2,2]
         ),
        self.specimen                       #[[0,1,3,2,2], [0,3,2,1,0], .... , n-1] For this case, n = generation_size
    ))

  def fitness(self, specimen): #Use params: self.params["max_weight"] and self.params["items"]
    
        comb = list(zip(specimen,params["items"])) #if specimen = [0,1,3,3,1] then [(0, Item[4,12]), (1, Item(2,2)),..(n-1)]
        #print(specimen)
        #print(sum(map(lambda x: x[0]*x[1].value,comb)))
        #print(sum(map(lambda x: x[0]*x[1].weight,comb)))
        if(sum(map(lambda x: x[0]*x[1].weight,comb))<= params["max_weight"]): # Join if only if backpack size is less or equal than 15
            return sum(map(lambda x: x[0]*x[1].value,comb))
        return (-math.inf)                                                   #In case that not, return -inf. This indicate backpack size is major than 15

  def is_converged(self):
    if any(self.fitness(specimen) >= self.params["fit_threshold"] for specimen in self.specimen):
      return True    # If exist any specimen with weight major or equal to expected.

    return False    #In other case.

  def get_fit(self):
    evaluations = self.fitness_all()

    max_evaluation = max(evaluations)

    max_index = evaluations.index(max_evaluation)

    return self.specimen[max_index], max_evaluation

  def fitness_all(self):
    #print(self.specimen)
    #print(list(map(self.fitness, self.specimen)))
    return list(map(self.fitness, self.specimen)) #Get all the evaluation fitness of the specimens. [10,20,36,...n-1]

  def select_specimen(self, specimen_evaluations):
    specimen_and_evaluations = list(zip(self.specimen, specimen_evaluations)) # [([0, 1, 1, 0, 2], 24),... (n-1)]

    specimen_and_evaluations.sort(key=lambda e: e[1], reverse = True)   #Order from largest to smallest. Descending order

    n_top = int(math.ceil(len(self.specimen) * params["select_top"]))   #Choosing the specimen that will pass to be mutated

    return list(map(lambda s: s[0], specimen_and_evaluations[:n_top]))  #Return the best specimens
  
  def mutate(self, specimen): #Use params: self.params["mutation_percentage"] and self.params["max_per_item"]
    
        n_digits = int(self.params["mutation_percentage"] * (len(specimen))) #Amount of item of the specimen to mutate. example: len specimen = 20 and percentage = 0,1 then 20*0,1 = 2. It will mutate two items of the specimen
        digit_indexes = random.sample(list(range(len(specimen))), n_digits) #Choosing n_digits random from a list that contain [0, 1, 2, 3, 4]. Maybe [0,4] if n_digits = 2, this are the index where it will change an item in the backpack
        mutated = specimen[:]  #Clone the specimen of the parameter. Example: [0, 1, 1, 0, 2] 

        for i in digit_indexes:             
            mutated[i] = random.choice(self.ALL_NUMBERS) #Choose an item according to the possible quantity in ALL_NUMBERS

        return mutated            #Return the specific specimen mutated. [0, 1, 1, 0, 2] then might return [2, 1, 1, 0, 1]

  def generate_children(self, selected_specimen):  
    mutated_specimen = [None] * len(self.specimen)

    for i in range(len(mutated_specimen)):
      mutated_specimen[i] = self.mutate(random.choice(selected_specimen))

    return mutated_specimen
  
  def run(self):
    generation_number = 1

    while generation_number <= self.params["max_generations"] and not self.is_converged():
      top_generation = self.get_fit()
      top_str = "".join(str(top_generation[0]))
      
      print(f"Generation #{generation_number}:\t{top_str}\t{top_generation[1]}")

      specimen_evaluations = self.fitness_all()
      selected_specimen = self.select_specimen(specimen_evaluations)
      
      self.specimen = self.generate_children(selected_specimen)
      
      generation_number += 1
    
    return self.get_fit()

In [90]:
class Item:
  def __init__(self, value, weight):
    self.value = value
    self.weight = weight

params = {
    "mutation_percentage": 0.4,
    "select_top": 0.3,
    "generation_size": 30,
    "fit_threshold": 35,
    "max_generations": 100,
    "max_weight": 15,
    "max_per_item": 4,
    "items": [Item(4, 12), Item(2, 2), Item(2, 1), Item(1, 1), Item(10,4)]
}

knapsack = KnapsackGenetic(params)
knapsack.run()

Generation #1:	[0, 0, 3, 1, 1]	17
Generation #2:	[0, 4, 3, 0, 1]	24
Generation #3:	[0, 3, 4, 0, 1]	24
Generation #4:	[0, 0, 3, 0, 2]	26
Generation #5:	[0, 2, 0, 0, 2]	24
Generation #6:	[0, 2, 0, 0, 2]	24
Generation #7:	[0, 1, 3, 2, 2]	30
Generation #8:	[0, 0, 3, 4, 2]	30
Generation #9:	[0, 0, 4, 1, 2]	29


([0, 0, 2, 1, 3], 35)